In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA



Загрузити датасет, що містить інформацію про відвідуваність музеїв в різні роки.

In [ ]:
from google.colab import files



uploaded = files.upload()


file_name = "museum_visitors.csv"


data = pd.read_csv(file_name, encoding='cp1251')


Saving museum_visitors.csv to museum_visitors.csv


Вивести перші 5 рядків датасету.

In [ ]:
data.head(5)

,Date,Avila Adobe,Firehouse Museum,Chinese American Museum,America Tropical Interpretive Center
0,2014-01-01,24778,4486,1581,6602
1,2014-02-01,18976,4172,1785,5029
2,2014-03-01,25231,7082,3229,8129
3,2014-04-01,26989,6756,2129,2824
4,2014-05-01,36883,10858,3676,10694


Провести попередній аналіз даних (визначити розмір датасету, тип даних, кількість пропусків).

In [ ]:
print("Table Size:", data.shape)
print("\nMissing Values:\n", data.isnull().sum())
print("\nColumn Types:\n", data.dtypes)

Table Size: (59, 5)

Missing Values:
 Date                                    0
Avila Adobe                             0
Firehouse Museum                        0
Chinese American Museum                 0
America Tropical Interpretive Center    0
dtype: int64

Column Types:
 Date                                    object
Avila Adobe                              int64
Firehouse Museum                         int64
Chinese American Museum                  int64
America Tropical Interpretive Center     int64
dtype: object


Перевірити наявність дублікатів. Якщо є, видалити їх і вивести розмір таблиці (без дублікатів).

In [ ]:

duplicates = data.duplicated()
num_duplicates = duplicates.sum()
print(f"\nNumber of duplicates: {num_duplicates}")

data_no_duplicates = data.drop_duplicates()


print("\nSize of the table without duplicates:")
print(data_no_duplicates.shape)



Number of duplicates: 0

Size of the table without duplicates:
(59, 5)


Про які роки містить інформацію датасет.


In [ ]:

unique_years = data_no_duplicates['Date'].unique()

print("Unique years in the dataset:")
print(unique_years)

Unique years in the dataset:
['2014-01-01' '2014-02-01' '2014-03-01' '2014-04-01' '2014-05-01'
 '2014-06-01' '2014-07-01' '2014-08-01' '2014-09-01' '2014-10-01'
 '2014-11-01' '2014-12-01' '2015-01-01' '2015-02-01' '2015-03-01'
 '2015-04-01' '2015-05-01' '2015-06-01' '2015-07-01' '2015-08-01'
 '2015-09-01' '2015-10-01' '2015-11-01' '2015-12-01' '2016-01-01'
 '2016-02-01' '2016-03-01' '2016-04-01' '2016-05-01' '2016-06-01'
 '2016-07-01' '2016-08-01' '2016-09-01' '2016-10-01' '2016-11-01'
 '2016-12-01' '2017-01-01' '2017-02-01' '2017-03-01' '2017-04-01'
 '2017-05-01' '2017-06-01' '2017-07-01' '2017-08-01' '2017-09-01'
 '2017-10-01' '2017-11-01' '2017-12-01' '2018-01-01' '2018-02-01'
 '2018-03-01' '2018-04-01' '2018-05-01' '2018-06-01' '2018-07-01'
 '2018-08-01' '2018-09-01' '2018-10-01' '2018-11-01']


Змінити назви столбців. Замінити великі букви на маленьки, пробіли на підкреслення (date, avila_adobe, firehouse_museum, chinese_american_museum, america_tropical_interpretive_center).

In [ ]:

data_no_duplicates.rename(columns={
    'Date': 'date',
    'Avila Adobe': 'avila_adobe',
    'Firehouse Museum': 'firehouse_museum',
    'Chinese American Museum': 'chinese_american_museum',
    'America Tropical Interpretive Center': 'america_tropical_interpretive_center'
}, inplace=True)

print(data_no_duplicates.head())


         date  avila_adobe  firehouse_museum  chinese_american_museum  \
0  2014-01-01        24778              4486                     1581   
1  2014-02-01        18976              4172                     1785   
2  2014-03-01        25231              7082                     3229   
3  2014-04-01        26989              6756                     2129   
4  2014-05-01        36883             10858                     3676   

   america_tropical_interpretive_center  
0                                  6602  
1                                  5029  
2                                  8129  
3                                  2824  
4                                 10694  


Обчислити середню кількість відвідувачів для кожного музею протягом всього періоду.

In [ ]:

data_no_duplicates['average_visitors'] = data_no_duplicates.mean(axis=1)


print("Розмір таблиці після додавання середньої кількості відвідувачів:", data_no_duplicates.shape)
print(data_no_duplicates.head())


Розмір таблиці після додавання середньої кількості відвідувачів: (59, 6)
         date  avila_adobe  firehouse_museum  chinese_american_museum  \
0  2014-01-01        24778              4486                     1581   
1  2014-02-01        18976              4172                     1785   
2  2014-03-01        25231              7082                     3229   
3  2014-04-01        26989              6756                     2129   
4  2014-05-01        36883             10858                     3676   

   america_tropical_interpretive_center  average_visitors  
0                                  6602           9361.75  
1                                  5029           7490.50  
2                                  8129          10917.75  
3                                  2824           9674.50  
4                                 10694          15527.75  


<ipython-input-21-849a9b040c7d>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data_no_duplicates['average_visitors'] = data_no_duplicates.mean(axis=1)


Знайти мінімальну та максимальну кількість відвідувачів для кожного музею за 2018 рік

In [ ]:

columns_2018 = ['date', 'avila_adobe', 'firehouse_museum', 'chinese_american_museum', 'america_tropical_interpretive_center']
data_2018 = data_no_duplicates[columns_2018]

min_visitors_2018 = data_2018.min()
max_visitors_2018 = data_2018.max()


print("Мінімальна кількість відвідувачів за 2018 рік:")
print(min_visitors_2018)
print("\nМаксимальна кількість відвідувачів за 2018 рік:")
print(max_visitors_2018)


Мінімальна кількість відвідувачів за 2018 рік:
date                                    2014-01-01
avila_adobe                                  14035
firehouse_museum                              3306
chinese_american_museum                       1073
america_tropical_interpretive_center          2824
dtype: object

Максимальна кількість відвідувачів за 2018 рік:
date                                    2018-11-01
avila_adobe                                  41242
firehouse_museum                             61192
chinese_american_museum                       7702
america_tropical_interpretive_center         13490
dtype: object


Визначити місяці з найвищою і найнижчою загальною кількістю відвідувачів серед усіх музеїв для 2015 року. Вивести назви місяців (не цифри).

Висновок
завдання 1:

У цьому завданні було проаналізовано та кластеризовано дані з файлу shopping_trends.csv. Після завантаження даних і перевірки розміру таблиці, пропусків і типів стовпців були обрані важливі числові характеристики, такі як "вік", "сума покупки (USD)", "рейтинг відгуків" і "попередні покупки".

Потім було застосовано метод масштабування, щоб переконатися, що числові ознаки були однаково масштабовані. Для визначення оптимальної кількості кластерів, в які можна було б ефективно згрупувати дані, використовувався метод ліктя. Оптимальна кількість кластерів була визначена як "точка ліктя" на графіку залежності значень кластерів від кількості кластерів.

Потім дані були згруповані за допомогою алгоритму K-середніх. Результати були візуалізовані за допомогою методу аналізу головних компонент (PCA), який зменшив розмірність даних і дозволив відобразити їх у двовимірному просторі.

Таким чином, алгоритм K-середніх був успішно використаний для аналізу та кластеризації даних, надаючи уявлення про структуру даних та групування, а метод PCA допомагає візуалізувати результати кластеризації, роблячи їх більш зрозумілими та легшими для інтерпретації.

висновок Завдання 2: Проаналізувавши дані про відвідуваність музеїв, було виявлено важливі залежності та тенденції. Набір даних містить інформацію за кілька років і з різних музеїв. Важливими кроками в обробці та аналізі даних є видалення дублікатів, розрахунок середньої кількості відвідувачів для кожного музею та виявлення екстремальних значень у певні роки.

При візуалізації даних розглядалися кореляційні теплові карти, лінійні графіки, що показують динаміку кількості відвідувачів, діаграми розсіювання для оцінки розподілу та гістограми кількості відвідувачів за місяць. Це дозволило отримати важливу інформацію про популярність окремих музеїв, тенденції у відвідуваності та взаємодію між різними музеями.

 Аналіз показав, які музеї є найпопулярнішими і як змінювалася кількість їхніх відвідувачів з роками.